# Comandos selenium para acessar o site

In [ ]:
from dotenv import load_dotenv
import os 
load_dotenv()

CPF_TALUDE = str(os.getenv("CPF_TALUDE"))
COD_UNIODONTO = str(os.getenv("COD_UNIODONTO"))
PASSWORD = str(os.getenv("PASSWORD"))

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

opts = Options()
opts.add_argument("--start-maximized")  # Inicia maximizado
opts.add_experimental_option("detach", True)

driver = webdriver.Chrome(options=opts)  # Selenium Manager resolve o driver


In [ ]:
driver.get("https://www.uniodontosc.coop.br/cooperados/index.php")

In [ ]:
from selenium.webdriver.common.by import By
from time import sleep
import os
import time
import requests
from urllib.parse import urljoin
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
sleep(3)

In [ ]:
cpf_bar = driver.find_element(By.ID, 'campoCpf')
sleep(1)
cod_bar = driver.find_element(By.ID, 'campoCodigo')
sleep(1)
paswd_bar = driver.find_element(By.ID, 'campoSenha')

In [ ]:
for numero in CPF_TALUDE:
    sleep(0.5)
    cpf_bar.send_keys(numero)


In [ ]:
cod_bar.send_keys(COD_UNIODONTO)

In [ ]:
paswd_bar.send_keys(PASSWORD)

In [ ]:
sleep(3)

In [ ]:
login_button = driver.find_element(By.XPATH, '/html/body/div[2]/section/div[3]/div/div/div/div/form/div/div[4]/div/div/button')

In [ ]:
login_button.click()

In [ ]:
sleep(1.5)

In [ ]:
close_add_button = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div/a')

In [ ]:
close_add_button.click()

In [ ]:
sleep(1.5)

In [ ]:
production_button = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[2]/div/div[4]/nav/ul/li[6]/a')

In [ ]:
production_button.click()

In [ ]:
sleep(0.5)

In [ ]:
lote_generation = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[2]/div/div[4]/nav/ul/li[6]/ul/li[1]/a')

In [ ]:
lote_generation.click()

In [ ]:
sleep(2)

In [ ]:
empresa_lote = driver.find_element(By.ID, 'empresaLote')

In [ ]:
empresa_lote.click()

In [ ]:
all_option = driver.find_element(By.XPATH, '/html/body/form/div/div/table/tbody/tr/td/table/tbody/tr[2]/td[3]/select/option[12]')

In [ ]:
all_option.click()

In [ ]:
month_year = driver.find_element(By.ID, 'mesAno')

In [ ]:

mesano = '102025'

In [ ]:
month_year.clear()
month_year.click()
sleep(1.5)
for n in mesano:
    sleep(1)
    month_year.send_keys(int(n))

In [ ]:
search = driver.find_element(By.ID, 'pesquisarGuia')

In [ ]:
search.click()

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
sleep(3)

In [ ]:
# Localizar a tabela
tabela = driver.find_element(By.XPATH, '//*[@id="tabelaListagem"]')

# Pegar todas as linhas
linhas = tabela.find_elements(By.XPATH, './/tbody/tr')

In [ ]:
from pathlib import Path

diretorio = Path('/Users/enzobarbi/Development/Projects/guias-uniodonto/fotos')

# Listar apenas arquivos
arquivos = [arquivo.name for arquivo in diretorio.iterdir() if arquivo.is_file()]

In [ ]:
# Criar um dicionário com protocolo como chave
protocolos_disponiveis = {}
for arquivo in arquivos:
    # Extrair o protocolo do nome do arquivo
    # Exemplo: "JANETE_BAUER_JACINTO - 5190363-XBE - 17102025 - 210,00 - GTO.jpg"
    partes = arquivo.split(' - ')
    if len(partes) >= 2:
        protocolo = partes[1].strip()  # "5190363-XBE"
        protocolos_disponiveis[protocolo] = arquivo

print("Protocolos disponíveis nos arquivos:")
print(protocolos_disponiveis)
print("-" * 50)

In [ ]:
ANEXO =partes[-1].split(".")[0]

In [ ]:
# Localizar a tabela
tabela = driver.find_element(By.XPATH, '//*[@id="tabelaListagem"]')
linhas = tabela.find_elements(By.XPATH, './/tbody/tr')

In [ ]:
# Percorrer as linhas e verificar se existe arquivo correspondente
for linha in linhas:
    celulas = linha.find_elements(By.TAG_NAME, 'td')
    
    if len(celulas) >= 5:
        protocolo = celulas[1].text.strip()  # Ex: "5240754-JFF"
        data = celulas[2].text.strip()
        beneficiario = celulas[3].text.strip()
        valor = celulas[4].text.strip()
        
        print(f"Verificando protocolo: {protocolo}")
        
        # Verificar se existe arquivo para este protocolo
        if protocolo in protocolos_disponiveis:
            print(f"✅ ENCONTRADO! Arquivo: {protocolos_disponiveis[protocolo]}")
            print(f"   Beneficiário: {beneficiario}")
            print(f"   Data: {data}")
            print(f"   Valor: {valor}")
            
            # CLICAR NA LINHA
            celulas[1].click()  # Clica no protocolo


            # localizar botões de anexo


            
            print(f"   >>> Clicou na linha do protocolo {protocolo}")
            print("-" * 50)
            
            # Se você quiser processar apenas um de cada vez, adicione um break
            break
        else:
            print(f"❌ NÃO ENCONTRADO - Protocolo {protocolo} sem arquivo correspondente")
            print("-" * 50)

In [ ]:
sleep(3)

In [ ]:
rx_button = driver.find_element(By.ID, "AnexarRx1")
gto_button = driver.find_element(By.ID, "AnexarRx2")

In [ ]:
if ANEXO == "GTO":
    gto_button.click()
else:
    print("Implementar o RX")

In [ ]:
import os
import time
import requests
from urllib.parse import urljoin
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
wait = WebDriverWait(driver, 10)

In [ ]:
arquivo_nome = protocolos_disponiveis[protocolo]
arquivo_path = os.path.join(diretorio, arquivo_nome)

In [ ]:
arquivo_nome

In [ ]:
arquivo_path

In [ ]:
print("  → Navegando para página de upload")
try:
    iframe = wait.until(EC.presence_of_element_located((
        By.CSS_SELECTOR, "iframe#TB_iframeContent, iframe[src*='imagens_lote_guias.php']"
    )))
    
    src = iframe.get_attribute("src")
    src_abs = urljoin(driver.current_url, src)
    driver.get(src_abs)
    print("  ✓ Navegação concluída")
except Exception as e:
    print(f"  ✗ Erro na navegação: {e}")

In [ ]:
# 3. Fazer upload usando requests
print(f"  → Fazendo upload: {arquivo_nome}")
url_atual = driver.current_url

if "controle=" not in url_atual:
    print(f"  ✗ Erro: URL não contém parâmetro 'controle'")
    driver.back()
    time.sleep(2)

codigo = url_atual.split("controle=")[1].split("&")[0]

try:
    cookie = driver.get_cookie('PHPSESSID')
    if not cookie or 'value' not in cookie:
        print(f"  ✗ Erro: Cookie PHPSESSID não encontrado")
        driver.back()
        time.sleep(2)
    
    phpsessid = cookie['value']
except Exception as e:
    print(f"  ✗ Erro ao obter cookie: {e}")
    driver.back()
    time.sleep(2)

url_upload = "https://www.sisoweb.coop.br/web/cooperados/upload.process.imagens.php"

headers = {
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'x-requested-with': 'XMLHttpRequest',
    'referer': url_atual
}

cookies = {
    'PHPSESSID': phpsessid,
    'loginCooperados': '1'
}

data = {
    'controle': codigo,
    'baixada': 'f',
    'liberada': 'f',
    'pendente': 'f'
}

try:
    with open(arquivo_path, 'rb') as f:
        files = {
            'files[]': (arquivo_nome, f, 'image/jpeg')
        }
        response = requests.post(
            url_upload, 
            data=data, 
            files=files, 
            headers=headers, 
            cookies=cookies, 
            timeout=120
        )
    
    if response.status_code == 200:
        print(f"  ✓ Upload realizado com sucesso")
        time.sleep(2)
        driver.refresh()
        time.sleep(2)
    else:
        print(f"  ✗ Erro no upload: Status {response.status_code}")
        driver.back()
        time.sleep(2)
        
except Exception as e:
    print(f"  ✗ Erro durante upload: {e}")
    driver.back()
    time.sleep(2)
    

In [ ]:
# Localiza diretamente a segunda opção do select com ID dinâmico
element = driver.find_element(By.XPATH, "//*[starts-with(@id, 'procedimento')]/option[2]")

In [ ]:
element.click()

In [ ]:
button_concluir = driver.find_element(By.XPATH, '//*[@id="btnEnviarArquivoImagem"]')

In [ ]:
button_concluir.click()

In [ ]:
def deletar_arquivo():
    os.remove(arquivo_path)

deletar_arquivo()


In [ ]:
driver.get("https://www.sisoweb.coop.br/web/cooperados/gerar.lote.php")


In [ ]:
#Fazer o loop novamente da tabela.